In [68]:
from abc_utils import *
import pandas as pd
from hmmlearn import hmm
import numpy as np
from sklearn.metrics import accuracy_score

In [98]:
train_set, train_lengths, val_set, _ = load_datasets()

In [113]:
def load_song(index):
    start_pos = train_lengths.iloc[:index].sum().item()
    return train_set.iloc[start_pos : start_pos + train_lengths.iloc[index].item()]

In [121]:
def load_songs(indices):
    end_positions = np.cumsum(train_lengths)
    positions = np.insert(end_positions, 0, np.array([0]))
    songs = []
    for i in indices:
        song = train_set.iloc[positions[i] : positions[i+1]]
        songs.append(song)
    # return songs and lengths
    return pd.concat(songs), train_lengths.iloc[indices, :].values.flatten()

song_indices = list(range(100))
songs, song_lengths = load_songs(song_indices)

In [122]:
# define global variables for number of chords and number of songs
NUM_CHORDS = 2
NUM_NOTES = 2
true_states, true_observations = dataframe_to_states(
    songs, 
    NUM_CHORDS, 
    NUM_NOTES
)

Processing states: 100%|██████████| 9052/9052 [00:00<00:00, 41269.29it/s]


In [123]:
transition_matrix, emission_probs, unique_states, unique_obs, states_to_index, observation_to_index = states_to_transition(true_states, true_observations)

In [124]:
observation_indices = np.array([int(observation_to_index[(o,)]) for o in true_observations]) 

In [125]:
model = hmm.CategoricalHMM(
    n_components=transition_matrix.shape[0],
    init_params=''
)

In [126]:
model.transmat_ = transition_matrix.T
model.emissionprob_ = emission_probs.T
start_probs = np.ones(transition_matrix.shape[0]) / transition_matrix.shape[0]
model.startprob_ = start_probs

print(model.transmat_.shape, model.emissionprob_.shape, model.startprob_.shape)

(3047, 3047) (3047, 39) (3047,)


Other tools:
- dynamax
  - runs on JAX
  - might have a more flexible structure
  - 

In [ ]:
likelihood, pred_states = model.decode(observation_indices.reshape(-1, 1))
pred_states

# get the actual chords from unique_states
states = unique_states[pred_states, :]
states

def chord_accuracy(full_pred: np.array, true_states: np.array, num_chords: int=NUM_CHORDS, num_notes: int=NUM_NOTES):
    '''
    Given the predicted matrix of states, compute the misclassification rate compared with the true_observations.
    Could be edited in the future to also compute the accuracy of our predicted note sequence.
    '''
    # obtain the actual predicted chords 
    pred_chords = full_pred[:, num_chords-1]
    true_chords = true_states[:len(pred_chords), num_chords-1]
    # obtain the accuracy
    acc = accuracy_score(true_chords, pred_chords)

    return acc

print(chord_accuracy(states, true_states))

In [ ]:
songs, song_lengths = load_songs([18])
true_states, true_observations = dataframe_to_states(
    songs, 
    NUM_CHORDS, 
    NUM_NOTES
)
observation_indices = np.array([int(observation_to_index[(o,)]) for o in true_observations]) 
likelihood, pred_states = model.decode(observation_indices.reshape(-1, 1))
states = unique_states[pred_states, :]
chord_accuracy(states, true_states)

Processing states: 100%|██████████| 115/115 [00:00<00:00, 31284.53it/s]


0.008695652173913044